In [222]:
import cPickle as pickle
from matplotlib import pyplot as plt
import os

# Grab Input Data
training_labels_file = '../finetuning/VOC_cropped_warped_train_small.txt'
data_files_path = 'rcnn_features/'

data = []
labels = []
data_files = []
with open(training_labels_file) as f:
    for line in f:
        data_file, label = line.split(' ')
        data_file = data_file.split('/')[-1].split('.')[0] + '.pi'
        try:
            data.append(pickle.load(open(data_files_path + data_file, 'rb')))
            labels.append(int(label))
            data_files.append(data_file)
        except IOError as e:
            pass
data = np.squeeze(np.array(data))
labels = np.array(labels)
data_files = np.array(data_files)
print data.shape, labels.shape, data_files.shape

(909, 200) (909,) (909,)


In [223]:
# Split data into training and test
num_vectors = data.shape[0]
test_ratio = .1
num_test = num_vectors * test_ratio

indices = np.random.permutation(num_vectors)
training_idx, test_idx = indices[num_test:], indices[:num_test]
training_data, test_data = data[training_idx,:], data[test_idx,:]
training_labels, test_labels = labels[training_idx,], labels[test_idx,]

print training_data.shape, test_data.shape
print training_labels.shape, test_labels.shape

(819, 200) (90, 200)
(819,) (90,)


In [224]:
# Define Classifier
from sklearn import svm, neighbors, linear_model, ensemble
# classifier = svm.LinearSVC()
# classifier = svm.SVC()
# classifier = neighbors.KNeighborsClassifier()
# classifier = linear_model.SGDClassifier()
classifier = ensemble.RandomForestClassifier()
# classifer = ensemble.AdaBoostClassifier()
# classifer = ensemble.GradientBoostingClassifier()
# classifer = ensemble.ExtraTreesClassifier()

In [225]:
% time
# Train Classifier
classifier.fit(training_data, training_labels);

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [226]:
# Test Classifier
predicted_labels = classifier.predict(test_data)
print classifier.score(test_data, test_labels)

0.677777777778


In [227]:
import pandas
wrong_test_idx = np.arange(test_labels.shape[0])[predicted_labels != test_labels]
wrong_data_files_idx = test_idx[wrong_test_idx]
wrong_data_files = data_files[wrong_data_files_idx]

wrong_predicted = predicted_labels[predicted_labels != test_labels]
wrong_true = test_labels[predicted_labels != test_labels]

print len(wrong_true), '/', len(test_labels)
print pandas.DataFrame({"Image Index": wrong_data_files, "Predicted": wrong_predicted, "Truth": wrong_true})

29 / 90
    Image Index  Predicted  Truth
0   pict_899.pi          0      3
1   pict_463.pi          0      3
2   pict_928.pi          3      1
3   pict_194.pi          0      3
4   pict_697.pi          0      1
5   pict_602.pi          1      0
6   pict_565.pi          0      1
7   pict_415.pi          0      1
8   pict_719.pi          3      0
9   pict_587.pi          0      3
10  pict_842.pi          0      4
11  pict_133.pi          0      2
12  pict_102.pi          0      1
13  pict_558.pi          0      3
14  pict_537.pi          0      3
15  pict_961.pi          0      1
16  pict_296.pi          0      2
17  pict_647.pi          0      3
18   pict_29.pi          0      1
19   pict_42.pi          0      2
20  pict_911.pi          0      3
21  pict_585.pi          0      1
22  pict_969.pi          0      3
23  pict_474.pi          0      1
24  pict_214.pi          0      3
25  pict_864.pi          0      3
26  pict_645.pi          0      3
27  pict_460.pi          0      4
28  pi

In [228]:
# Labels 0-4 = person, bike, bus, car, motorbike